**FINANCIAL FRAUD DETECTION**
====

# Part 1: Reading the input data file and storing to DataFrame

In [1]:
from pyspark.sql.types import *
import sys
import os
sqlContext = SQLContext(sc)

###Read input file and convert it to RDD.
1. Read from file system
2. split columns
3. filter header
4. filter empty rows

In [2]:
filename = '/vagrant/PS_20174392719_1491204439457_log.csv'

def convertToTransactionSchema(arr):
    res = arr
    res[0] = int(arr[0]) # step
    res[2] = float(arr[2]) # amount
    res[4] = float(arr[4]) # old balance
    res[5] = float(arr[5]) # new balance
    res[7] = float(arr[7]) # old balance destination
    res[8] = float(arr[8]) # new balance destination
    res[9] = int(arr[9]) # is fraud
    res[10] = int(arr[10]) # is flagged fraud
    return res

transactionsRDD = (sc.textFile(filename)
       .map(lambda line: line.split(","))
       .filter(lambda line: line[0] != "step")
       .filter(lambda line: len(line)>1)
       .map(convertToTransactionSchema))
transactionsRDD.cache()

PythonRDD[2] at RDD at PythonRDD.scala:43

###Convert transactionsRDD into DataFrame and cache it.

In [30]:
transactionSchema = StructType([
    StructField("step", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("amount", FloatType(), True),
    StructField("nameOrig", StringType(), True),
    StructField("oldbalanceOrig", FloatType(), True),
    StructField("newbalanceOrig", FloatType(), True),
    StructField("nameDest", StringType(), True),
    StructField("oldbalanceDest", FloatType(), True),
    StructField("newbalanceDest", FloatType(), True),
    StructField("isFraud", IntegerType(), True),
    StructField("isFlaggedFraud", IntegerType(), True)])

transactionsDF = sqlContext.createDataFrame(transactionsRDD, transactionSchema)
transactionsDF.cache()
print transactionsDF.take(2)

[Row(step=1, type=u'PAYMENT', amount=9839.6396484375, nameOrig=u'C1231006815', oldbalanceOrig=170136.0, newbalanceOrig=160296.359375, nameDest=u'M1979787155', oldbalanceDest=0.0, newbalanceDest=0.0, isFraud=0, isFlaggedFraud=0), Row(step=1, type=u'PAYMENT', amount=1864.280029296875, nameOrig=u'C1666544295', oldbalanceOrig=21249.0, newbalanceOrig=19384.720703125, nameDest=u'M2044282225', oldbalanceDest=0.0, newbalanceDest=0.0, isFraud=0, isFlaggedFraud=0)]


###Check if the DataFrame is cached.

In [4]:
transactionsDF.is_cached

True

# Part 2: Data Cleaning

###The purpose of data cleaning is to check if there is any invalid or missing values, and eliminate those reords.

###Print the datatype for each column.

In [5]:
print transactionsDF.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: float (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrig: float (nullable = true)
 |-- newbalanceOrig: float (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: float (nullable = true)
 |-- newbalanceDest: float (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)

None


###We need to check each column for invalid values. The set of invalid values depends on the data type of the columns:
1. For columns with string type: empty or null values.
2. Otherwise: null or minus values.

###Firstly, we need to get columns that has 'string' datatype and store it in a variable.

In [6]:
stringCols = []
for (colName, colType) in transactionsDF.dtypes:
    if(colType == 'string'):
        stringCols.append(colName)
        
print stringCols

['type', 'nameOrig', 'nameDest']


###Then, we count the number of invalid values in each column and show the result.

In [7]:
from pyspark.sql.functions import *
from pyspark.sql import *
#from IPython.display import display

def isInvalidValue(colName):
    print(colName)
    if colName in stringCols:
        return (col(colName).isNull() | (col(colName) == ''))
    else:
        return (col(colName).isNull() | (col(colName) < 0))

transactionsDF.select([count(when(isInvalidValue(c), c)).alias(c) for c in transactionsDF.columns]).show()

step
type
amount
nameOrig
oldbalanceOrig
newbalanceOrig
nameDest
oldbalanceDest
newbalanceDest
isFraud
isFlaggedFraud
+----+----+------+--------+--------------+--------------+--------+--------------+--------------+-------+--------------+
|step|type|amount|nameOrig|oldbalanceOrig|newbalanceOrig|nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+----+------+--------+--------------+--------------+--------+--------------+--------------+-------+--------------+
|   0|   0|     0|       0|             0|             0|       0|             0|             0|      0|             0|
+----+----+------+--------+--------------+--------------+--------+--------------+--------------+-------+--------------+



###As shown, the dataset does not contain any invalid values so we do not need to elminate any records.

## Part 3: Exploring the dataset

###First, we would explore some basic information in the dataset:

1. Number of fraudulent transactions
2. Types of the fraudulent transactions
3. What determines whether the feature isFlaggedFraud gets set or not?
4. Statistics of each column
5. Correllation analysis

###1. Number and percentage of fraudulent transactions and show the records

In [10]:
fraudulentTransactions = transactionsDF.filter(transactionsDF.isFraud == 1)
fraudulentTransactions.show()
print "The total number of fraudulent records = ", fraudulentTransactions.count()
print "The percentage of fraudulent records = %f%%" % ( float(fraudulentTransactions.count())/transactionsDF.count()*100)

+----+--------+---------+-----------+--------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrig|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+---------+-----------+--------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1|TRANSFER|    181.0|C1305486145|         181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|    181.0| C840083671|         181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1|TRANSFER|   2806.0|C1420196421|        2806.0|           0.0| C972765878|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   2806.0|C2101527076|        2806.0|           0.0|C1007251739|       26202.0|           0.0|      1|             0|
|   1|TRANSFER|  20128.0| C137533655|       20128.0|           0.0|C1

###2. Types of the fraudulent transactions

In [11]:
fraudulentTransactions.groupBy(fraudulentTransactions.type).count().show()

+--------+-----+
|    type|count|
+--------+-----+
|CASH_OUT| 4116|
|TRANSFER| 4097|
+--------+-----+



###We can see there are two fradulent types of transactions, CASH_OUT and TRANSFER

###3. What determines whether the feature isFlaggedFraud gets set or not?

## Part 4: Building up the fraud detection model

### To make the string columns categorical, they should be casted into double datatype first. 

In [50]:
categoricalTransactionsDF = transactionsDF.select([col(c).cast("double").alias(c) for c in transactionsDF.columns])
categoricalTransactionsDF.printSchema()
categoricalTransactionsDF.take(2)

root
 |-- step: double (nullable = true)
 |-- type: double (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: double (nullable = true)
 |-- oldbalanceOrig: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: double (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: double (nullable = true)
 |-- isFlaggedFraud: double (nullable = true)



[Row(step=1.0, type=None, amount=9839.6396484375, nameOrig=None, oldbalanceOrig=170136.0, newbalanceOrig=160296.359375, nameDest=None, oldbalanceDest=0.0, newbalanceDest=0.0, isFraud=0.0, isFlaggedFraud=0.0),
 Row(step=1.0, type=None, amount=1864.280029296875, nameOrig=None, oldbalanceOrig=21249.0, newbalanceOrig=19384.720703125, nameDest=None, oldbalanceDest=0.0, newbalanceDest=0.0, isFraud=0.0, isFlaggedFraud=0.0)]

In [49]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


labelIndexer = StringIndexer(inputCol="isFraud", outputCol="isFraudLabel").fit(categoricalTransactionsDF)

featuresCols = categoricalTransactionsDF.columns
featuresCols.remove('isFraud')

featureAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures").transform(categoricalTransactionsDF)
featureIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=5).fit(categoricalTransactionsDF)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

randomForest = RandomForestClassifier(labelCol="isFraudLabel", featuresCol="features")
pipeline = Pipeline(stages=[labelIndexer, featureAssembler, featureIndexer, randomForest])
model = pipeline.fit(trainingData)


IllegalArgumentException: u'Field "rawFeatures" does not exist.'

###Spilting the dataset

In [26]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils

trainingData, testData = transactionsRDD.randomSplit([0.7, 0.3])
print "There are %d training reords and %d test records." % (trainingData.count(), testData.count())
print "There are %d fraudulent records in the test data." % (testData.filter(transactionsDF.isFraud == 1))

There are 4455162 training reords and 1907458 test records.


## Part 5: Evaluating the fraud detection model

## Part 6: Analyzing the fraud detection model